In [1]:
import pandas as pd
import numpy as np
from helpers import *
from sklearn.linear_model import Ridge

In [11]:
# num_users, num_movies = len(rating),len(rating.columns)

# user_features = pd.DataFrame(np.random.rand(num_users,num_features),index=range(1,num_users+1),columns=range(1,num_features+1))
# movie_features = pd.DataFrame(np.random.rand(num_movies,num_features),index=range(1,num_movies+1),columns=range(1,num_features+1))

# user_ft_data = train_df.set_index('Movie').join(Z).sort_values('User').set_index('User')
# movie_ft_data = train_df.set_index('User').join(W).sort_values('Movie').set_index('Movie')

# user_ft_data.loc[1,:].head(1)

# user_ft_data.loc[1,user_ft_data.columns!='Rating']

In [67]:
num_users, num_movies = len(rating),len(rating.columns)
num_features = 20
lambda_ = 19
Z = pd.DataFrame(np.random.rand(num_movies,num_features),index=range(1,num_movies+1),columns=range(1,num_features+1))
user_ft_data = train_df.set_index('Movie').join(Z).sort_values('User').set_index('User')

In [78]:
y = user_ft_data.loc[1,['Rating']]

In [72]:
X = user_ft_data.loc[1,user_ft_data.columns!='Rating']

In [79]:
y.shape

(23, 1)

In [81]:
model = Ridge(fit_intercept=False,alpha = lambda_)

In [82]:
model.fit(X,y)

Ridge(alpha=19, copy_X=True, fit_intercept=False, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [83]:
X

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
User,,,,,,,,,,,,,,,,,,,,
1,0.977188,0.485274,0.299669,0.708525,0.068052,0.832719,0.332394,0.532197,0.623903,0.983963,0.290706,0.995982,0.707367,0.814075,0.341013,0.791500,0.385896,0.881090,0.818645,0.964859
1,0.605626,0.334609,0.564385,0.431611,0.695395,0.092084,0.894238,0.991743,0.125370,0.466625,0.177574,0.335416,0.297377,0.222259,0.686836,0.808933,0.496626,0.828284,0.674090,0.357236
1,0.148774,0.381736,0.582270,0.433496,0.178728,0.729442,0.193216,0.419295,0.504761,0.677814,0.597270,0.093942,0.285378,0.847904,0.964410,0.039940,0.904157,0.318355,0.584755,0.425729
1,0.148110,0.526497,0.716981,0.888905,0.939102,0.343697,0.005290,0.953637,0.932518,0.588183,0.375558,0.862390,0.452954,0.302543,0.058730,0.838111,0.184093,0.183810,0.342859,0.663493
1,0.984367,0.433607,0.973879,0.278037,0.440558,0.521405,0.808529,0.743164,0.359727,0.746164,0.120492,0.267966,0.740716,0.014714,0.639517,0.141499,0.636257,0.044337,0.907827,0.072300
1,0.820141,0.398979,0.754043,0.782818,0.715143,0.104568,0.883461,0.383213,0.222894,0.627187,0.723570,0.300603,0.095992,0.522198,0.365420,0.385059,0.583926,0.049471,0.989165,0.951339
1,0.755299,0.215064,0.962980,0.567268,0.090048,0.385628,0.571608,0.582896,0.153584,0.639955,0.792841,0.710972,0.990411,0.724153,0.259976,0.822954,0.751740,0.575541,0.628174,0.409665
1,0.360951,0.774505,0.533710,0.901826,0.163171,0.278553,0.593856,0.624665,0.240915,0.845553,0.969605,0.689218,0.183643,0.367334,0.671203,0.616969,0.086277,0.033366,0.498438,0.120839
1,0.233966,0.452159,0.785007,0.539748,0.225873,0.933491,0.701105,0.314620,0.028071,0.000546,0.427393,0.661224,0.854445,0.199293,0.610393,0.188128,0.287962,0.865881,0.471348,0.320644


In [24]:
def compute_rmse_rr(pred, rating):
    err = rating - pred
    mse = err**2
    return np.sqrt(np.mean(mse.mean(skipna=True)))

In [18]:
def update_user_features(W,Z,train_df,lambda_,num_users):
    user_ft_data = train_df.set_index('Movie').join(Z).sort_values('User').set_index('User')
    model = Ridge(fit_intercept=False,alpha = lambda_)
    
    for i in range(1,num_users+1):
        X = user_ft_data.loc[i,user_ft_data.columns!='Rating']
        y = user_ft_data.loc[i,['Rating']]
        model.fit(X,y)
        W.loc[i,:] = model.coef_
    return W


def update_movie_features(W,Z,train_df,lambda_,num_movies):
    movie_ft_data = train_df.set_index('User').join(W).sort_values('Movie').set_index('Movie')
    model = Ridge(fit_intercept=False,alpha = lambda_)
    
    for i in range(1,num_movies+1):
        X = movie_ft_data.loc[i,movie_ft_data.columns!='Rating']
        y = movie_ft_data.loc[i,['Rating']]
        model.fit(X,y)
        Z.loc[i,:] = model.coef_
    return Z

In [26]:
def MF_RR(train_tr, rating,num_features,lambda_,iterations=20):
    """Alternating Least Squares (ALS) algorithm."""
    # define parameters
    stop_criterion = 1e-5
    change = 1
    error_list = [0, 0]
    it = 0
 
    num_users, num_movies = len(rating),len(rating.columns)
    # init matrix
    user_features = pd.DataFrame(np.random.rand(num_users,num_features),index=range(1,num_users+1),columns=range(1,num_features+1))
    movie_features = pd.DataFrame(np.random.rand(num_movies,num_features),index=range(1,num_movies+1),columns=range(1,num_features+1))
    
    train_rmse = 0
    W = user_features.copy()
    Z = movie_features.copy()
    # start ALS
    while(it < iterations):
        W = update_user_features(W,Z,train_tr,lambda_,num_users)
        
        Z = update_movie_features(W,Z,train_tr,lambda_,num_movies)
        pred = W.dot(Z.T)
        pred[pred > 5] = 5
        pred[pred < 1] = 1
        train_rmse = compute_rmse_rr(pred,rating)
        print("MF-RR training RMSE : {err}".format(err=train_rmse))
        error_list.append(train_rmse)
        change = np.fabs(error_list[-1] - error_list[-2])
        if (change < stop_criterion):
            print("Converge!")
            break;
            
        it += 1
        
#     print("MF-RR Final training RMSE : {err}".format(err=train_rmse))
    return W,Z

In [57]:
def mf_rr_algo(model,train_df,test_df):
    
    num_features = 20
    lambda_ = 19
    
    rating = train_df.pivot(index="User",columns="Movie",values="Rating")
    
    user_features,movie_features = MF_RR(train_df,rating,num_features,lambda_,iterations=20)
    
    pred = user_features.dot(movie_features.T)
    pred[pred > 5] = 5
    pred[pred < 1] = 1
    
    test_user = test_df['User'].values
    test_movie = test_df['Movie'].values
    
    pred_test =[]
    for user,movie in zip(test_user,test_movie):
        pred_rating = pred.loc[user,movie]
        pred_test.append(pred_rating)
        
    pred_test = np.asarray(pred_test)

    test_ret = test_df.copy()
    test_ret.drop(columns='Rating',inplace=True)
    test_ret['Rating'] = pred_test
    prediction = test_ret.copy()
#     prediction = create_csv_submission(test_ret)
    
    return prediction

## Test MFRR algo

In [54]:
train_dataset = "./data/data_train.csv"
test_dataset = "./data/sampleSubmission.csv"

train_df = load_dataset(train_dataset)
test_df = load_dataset(test_dataset)

[load_dataset] Valid: (1176952, 3)
[load_dataset] Valid: (1176952, 3)


In [55]:
train_tr, test_tr = split_dataset(train_df,p_test=0.5)

[split_dataset] Valid: (1176952, 3)


In [62]:
train_tr

,User,Movie,Rating
0,2135,741,4
1,6850,357,2
2,4393,440,3
3,365,42,2
4,9660,546,5
5,8723,802,5
6,9380,716,4
7,1187,191,5
8,6820,788,3
9,6713,358,5


In [58]:
prediction = mf_rr_algo("",train_tr,test_tr)

MF-RR training RMSE : 0.9888417362091013
MF-RR training RMSE : 0.9756442891414643
MF-RR training RMSE : 0.9648709136415702
MF-RR training RMSE : 0.9582898528767403
MF-RR training RMSE : 0.9539443183324987
MF-RR training RMSE : 0.95109905813012
MF-RR training RMSE : 0.9493085565853918
MF-RR training RMSE : 0.9482580792515908
MF-RR training RMSE : 0.9477215880370957
MF-RR training RMSE : 0.9475373528892748
MF-RR training RMSE : 0.9475919504153082
MF-RR training RMSE : 0.9478056930300627
MF-RR training RMSE : 0.9481229623670366
MF-RR training RMSE : 0.9485047767673412
MF-RR training RMSE : 0.9489231249984413
MF-RR training RMSE : 0.9493588556197821
MF-RR training RMSE : 0.9497977406294663
MF-RR training RMSE : 0.9502297963659893
MF-RR training RMSE : 0.9506478322941487
MF-RR training RMSE : 0.9510469817429116


In [60]:
prediction.head()

,User,Movie,Rating
0,2135,741,3.845605
1,6850,357,2.125287
2,4393,440,3.244223
3,365,42,3.354287
4,9660,546,3.764170


## Debug for each

In [65]:
train_dataset = "./data/data_train.csv"
test_dataset = "./data/sampleSubmission.csv"

train_df = load_dataset(train_dataset)
test_df = load_dataset(test_dataset)
train_tr, test_tr = split_dataset(train_df)
# train_tr = train_tr.pivot(index="User",columns="Movie",values="Rating").fillna(0)
# test_tr = test_tr.pivot(index="User",columns="Movie",values="Rating").fillna(0)

[load_dataset] Valid: (1176952, 3)
[load_dataset] Valid: (1176952, 3)
[split_dataset] Valid: (1176952, 3)


In [66]:
rating = train_df.pivot(index="User",columns="Movie",values="Rating")

In [27]:
num_features = 20
lambda_ = 19
user_features,movie_features = MF_RR(train_tr,rating,num_features,lambda_,iterations=20)

MF-RR training RMSE : 0.9962715663977991
MF-RR training RMSE : 0.9821503744810715
MF-RR training RMSE : 0.9715191976415806
MF-RR training RMSE : 0.9648483388734127
MF-RR training RMSE : 0.9609606277479182
MF-RR training RMSE : 0.958657229973747
MF-RR training RMSE : 0.9572381437650561
MF-RR training RMSE : 0.9563431546319633
MF-RR training RMSE : 0.9557697728835788
MF-RR training RMSE : 0.9554186642932773
MF-RR training RMSE : 0.9552258745111935
MF-RR training RMSE : 0.9551474234591418
MF-RR training RMSE : 0.9551523752923804
Converge!


In [28]:
pred = user_features.dot(movie_features.T)
pred[pred > 5] = 5
pred[pred < 1] = 1

In [31]:
test_user = test_tr['User'].values
test_movie = test_tr['Movie'].values

In [43]:
pred_test =[]
for user,movie in zip(test_user,test_movie):
    pred_rating = pred.loc[user,movie]
    pred_test.append(pred_rating)
pred_test = np.asarray(pred_test)
pred_test[pred_test > 5] = 5
pred_test[pred_test < 1] = 1
test_ret = test_tr.copy()
test_ret['Pred'] = pred_test

In [45]:
err = test_ret['Rating']-test_ret['Pred']
mse = err**2
rmse = np.sqrt(mse.mean())

In [46]:
rmse

1.0082348576201594

## For Blend Test

In [64]:
from produce_predict_csv import mfrr

ModuleNotFoundError: No module named 'spotlight'